In [2]:
import cv2
import os
import numpy as np
import pandas as pd
from skimage import io
from PIL import Image

def classify_faces(source_folder, output_folder):
    # Load the DataFrame from the JSON file
    df = pd.read_json(source_folder)
    face_encodings = {}
    face_groups = {}

    for index in range(len(df)):
        # Get face encoding from the DataFrame
        vector = np.array(df['image_encode'].iloc[index], dtype=np.uint8)
        face_encoding = df['encode'].iloc[index]
        
        if len(face_encoding):
            # Check if there's a folder for this person
            person_folder = f"Person_{len(face_encodings) + 1}"
            for person, encoding in face_encodings.items():
                face_encoding_np = np.array(face_encoding)
                encoding_np = np.array(encoding)

                # Compare encodings to determine if it belongs to the same person
                if cv2.norm(face_encoding_np, encoding_np, cv2.NORM_L2) < 0.4:
                    person_folder = person
                    break

            if person_folder not in face_encodings:
                face_groups[person_folder] = []
                # If there's no folder for this person, create a new one
                face_encodings[person_folder] = face_encoding

            face_groups[person_folder].append(df['name'].iloc[index])

            if vector.shape[2] == 1:
                vector = vector.squeeze()

            # saved_image_path = os.path.join(person_folder, f"{df['name'].iloc[index]}.png")
            # Image.fromarray(vector).save(saved_image_path)
    return face_groups
   


  


In [3]:
source_folder = "crop_face_encode.json"
output_folder = "./similar_faces_encode/"

face_groups = classify_faces(source_folder, output_folder)

In [10]:
with open('graph_nodes.txt', 'w') as node:
  name_file_pairs = []
  for person_folder, names in face_groups.items():
        print(f"Group: {person_folder}")
        files = []
        
        for name in names:
            filename, extension = os.path.splitext(name)
      
              # Remove "_face_i" from the filename
            filename_without_suffix = filename.split('_face')[0]
            image = filename_without_suffix+extension
            node.write(filename+'.png,'+ '\n')
            files.append(image)
        for file in files:
            node.write(file+','+ '\n')
            name_file_pairs.append((file,filename+'.png')) 
  with open('graph_edges.txt', 'w') as egdes:
    for pair in name_file_pairs:
        egdes.write(repr(pair) + '\n')


Group: Person_1
Group: Person_2
Group: Person_3
Group: Person_4
Group: Person_5
Group: Person_6
Group: Person_7
Group: Person_8
Group: Person_9
Group: Person_10
Group: Person_11
Group: Person_12
Group: Person_13
Group: Person_14
Group: Person_15
Group: Person_16
Group: Person_17
Group: Person_18
Group: Person_19
Group: Person_20
Group: Person_21
Group: Person_22
Group: Person_23
Group: Person_24
Group: Person_25
Group: Person_26
Group: Person_27
Group: Person_28
Group: Person_29
Group: Person_30
Group: Person_31
Group: Person_32
Group: Person_33
Group: Person_34
Group: Person_35
Group: Person_36
Group: Person_37
Group: Person_38
Group: Person_39
Group: Person_40
Group: Person_41
Group: Person_42
Group: Person_43
Group: Person_44
Group: Person_45
Group: Person_46
Group: Person_47
Group: Person_48
Group: Person_49
Group: Person_50
Group: Person_51
Group: Person_52
Group: Person_53
Group: Person_54
Group: Person_55
Group: Person_56
Group: Person_57
Group: Person_58
Group: Person_59
Group: